In [1]:
import os
from dotenv import load_dotenv
import csv
import pandas as pd
from pinecone import Pinecone, ServerlessSpec
from pinecone.exceptions import PineconeApiException 
from langchain.embeddings.openai import OpenAIEmbeddings


/Users/franciscomeyo/Library/Caches/pypoetry/virtualenvs/llm-testing-P8xkvbMm-py3.11/lib/python3.11/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
## Get env variables
# Get the current working directory
current_directory = os.getcwd()
# Construct the path to the .env file in the parent directory
env_path = os.path.join(current_directory, '..', '.env')
# Load the environment variables from the .env file
load_dotenv(dotenv_path=env_path)
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')

In [3]:
def csv_to_dict(csv_file_path):
    """
    Convert a CSV file into a dictionary where the key is the first column.

    Args:
        csv_file_path (str): The path to the CSV file.

    Returns:
        dict: A dictionary representation of the CSV data with the first column as keys.
    """
    result_dict = {}
    
    # Open and read the CSV file
    with open(csv_file_path, mode='r', newline='') as file:
        reader = csv.DictReader(file)
        
        # Convert each row into a dictionary item
        for row in reader:
            key = row[reader.fieldnames[0]]  # Use the first column as the key
            result_dict[key] = {field: row[field] for field in reader.fieldnames[1:]}  # Use remaining columns as values
    
    return result_dict

def dict_to_csv(data_dict, csv_file_path):
    """
    Convert a dictionary to a DataFrame and save it to a CSV file.

    Args:
        data_dict (dict): The dictionary to be converted to a CSV file.
        csv_file_path (str): The path where the CSV file will be saved.

    Returns:
        None
    """
    # Convert the dictionary to a DataFrame
    df = pd.DataFrame.from_dict(data_dict, orient='index')
    
    # Save the DataFrame to a CSV file
    df.to_csv(csv_file_path)
    
    print(f"Data has been written to {csv_file_path}")

def select_pinecone_index(pc, index_name):

  # List current indexes to ensure the check is accurate
  current_indexes = pc.list_indexes()

  if index_name not in current_indexes:
      try:
          pc.create_index(
              name=index_name,
              dimension=1536, # 1536 dim of text-embedding-ada-002
              metric="cosine", # Replace with your model metric
              spec=ServerlessSpec(
                  cloud="aws",
                  region="us-east-1"
              ) 
          )
      except PineconeApiException as e:
          if "ALREADY_EXISTS" in str(e):
              print(f"Index '{index_name}' already exists.")
          else:
              raise
  else:
      print(f"Index '{index_name}' already exists.")

  return pc.Index(index_name)

def upsert_embeddings_to_pinecone(index, data_dict, batch_size=100):
    
    """
    Generate embeddings for each key in a dictionary and upsert them to a Pinecone index.

    Args:
        index (object): The Pinecone index to upsert vectors to.
        data_dict (dict): The input dictionary containing data. 
                          Each key should be the text to embed, 
                          and the value should contain an 'id' field.
        embed_function (function): The embedding function to use for generating embeddings.
        batch_size (int): The number of vectors to upsert in each batch. Default is 100.
    """
    # embeddings setup using OpenAI
    model_name = 'text-embedding-ada-002'
    embedder = OpenAIEmbeddings(
        model=model_name,
        openai_api_key=OPENAI_API_KEY
    )
    
    # Create lists to store embeddings, metadata, and IDs
    embeddings_list = []
    metadata_list = []
    ids = []

    # Generate embeddings for each key in the data dictionary
    for key in data_dict.keys():
        vector = embedder.embed_query(key)
        embeddings_list.append(vector)
        metadata_list.append({'name': key})  # Store the name as metadata
        ids.append(data_dict[key]['id'])

    # Batch and upsert embeddings with metadata
    for i in range(0, len(embeddings_list), batch_size):
        batch_ids = ids[i:i + batch_size]
        batch_vectors = embeddings_list[i:i + batch_size]
        batch_metadata = metadata_list[i:i + batch_size]
        
        # Combine ids, vectors, and metadata into a single iterable
        vectors_with_metadata = [
            (batch_ids[j], batch_vectors[j], batch_metadata[j])
            for j in range(len(batch_ids))
        ]
        
        # Upsert vectors with metadata to the Pinecone index
        index.upsert(vectors=vectors_with_metadata)

def find_closest_matches(source_index_name, target_index_name, data_dict, pinecone_client, top_k=1):
    """
    Query the Pinecone source index for each embedding and find the closest match in the target index.

    Args:
        source_index_name (str): The name of the Pinecone index containing the source embeddings.
        target_index_name (str): The name of the Pinecone index to search for matching vectors.
        data_dict (dict): A dictionary containing data with an 'id' field for each item.
        pinecone_client (object): The initialized Pinecone client object.
        top_k (int): The number of top matches to retrieve. Default is 1.

    Returns:
        dict: The updated data dictionary with matched vendor names and scores.
    """
    # Initialize indexes once outside the loop
    source_index = pinecone_client.Index(source_index_name)
    target_index = pinecone_client.Index(target_index_name)

    # Query the Pinecone index for each embedding and print the results
    for key, values in data_dict.items():
        # Get the item ID and fetch the corresponding vector from source index
        item_id = values.get('id')
        vector_data = source_index.fetch([item_id])['vectors'][item_id]

        # Get the vector values
        vector = vector_data['values']
        
        # Query the target index for the closest match
        matching_result = target_index.query(vector=vector, top_k=top_k, include_metadata=True)
        
        # Get the top match
        match = matching_result['matches'][0]
        
        # Extract metadata and score
        name = match['metadata']['name']
        score = match['score']
        
        # Update data_dict with the matching name and score
        data_dict[key]['matched_vendor'] = name
        data_dict[key]['score'] = score

        print(f"Closest match: {name} with a score of {score}")
    
    return data_dict


In [20]:
# Load csv files into dictionaries
csv_file_path = 'companies_prospects.csv'
vendors_dict_source = csv_to_dict(csv_file_path)
# csv_file_path = 'companies_bob.csv'
# vendors_dict_target = csv_to_dict(csv_file_path)

In [9]:
vendors_dict_source

{'Midcap Financial Services Capital Management (Apollo)': {'id': '1'},
 'MidOcean Partners': {'id': '2'},
 'MKP Capital Managmenet': {'id': '3'},
 'Moab Capital Partners LLC': {'id': '4'},
 'Mockingbird Credit Partners': {'id': '5'},
 'Monomoy Capital Partners (MCP Funds)': {'id': '6'},
 'Monroe Capital Advisors': {'id': '7'},
 'Mount Kellett Cap. Mgmt (Fortress)': {'id': '8'},
 'MSD Capital': {'id': '9'},
 'Mudrick Capital Management': {'id': '10'},
 'Mycor Capital': {'id': '11'},
 'Nearwater Capital': {'id': '12'},
 'New Mountain Capital': {'id': '13'},
 'New York Life Investment Management': {'id': '14'},
 'Newfleet Asset Management': {'id': '15'},
 'Newport Global Advisors': {'id': '16'},
 'NewStar Capital': {'id': '17'},
 'Nokota Management': {'id': '18'},
 'Northwoods Capital (CLOs by Angelo Gordon)': {'id': '19'},
 'Nuveen (Symphony Asset Management)': {'id': '20'},
 'NXT Capital Investment Advisors': {'id': '21'},
 'OFS Capital Management': {'id': '22'},
 'Olympus Peak Asset Ma

In [10]:
# Initialize Pinecone client
pc = Pinecone(api_key=PINECONE_API_KEY)

In [18]:
# Fetch the list of all indexes
current_indexes = pc.list_indexes()

# Display the list of indexes
print(current_indexes)


{'indexes': [{'deletion_protection': 'disabled',
              'dimension': 1536,
              'host': 'masterworks-282-llc-jean-michel-basquiat-poll-vf62p9b.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'masterworks-282-llc-jean-michel-basquiat-poll',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}},
             {'deletion_protection': 'disabled',
              'dimension': 1536,
              'host': 'companies-bob-vf62p9b.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'companies-bob',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}},
             {'deletion_protection': 'disabled',
              'dimension': 1536,
              'host': 'vendors-airtable-vf62p9b.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosi

In [19]:
# Delete the vendors-source index
pc.delete_index("companies-prospects")

# Delete the vendors-target index
# pc.delete_index("vendors-target")

# print("Indexes 'vendors-source' and 'vendors-target' have been deleted.")


In [21]:
# Embed and upsert the source and target dictionaries to Pinecone
index_name = "companies-prospects"
index = select_pinecone_index(pc, index_name)
upsert_embeddings_to_pinecone(index, vendors_dict_source, batch_size=100)
# index_name = "companies-bob"
# index = select_pinecone_index(pc, index_name)
# upsert_embeddings_to_pinecone(index, vendors_dict_target, batch_size=100)


In [22]:
# Find the closest matches between the source and target vector databases
matches_dict = find_closest_matches('companies-prospects', 'companies-bob', vendors_dict_source, pc)


Closest match: 683 Capital Management, LLC with a score of 0.908032835
Closest match: 683 Capital Management, LLC with a score of 0.888557076
Closest match: Blackstone Alternative Credit Advisors, with a score of 0.868887842
Closest match: Aberdeen Corporate Services Limited with a score of 0.910324872
Closest match: PropCap Advisors, LLC with a score of 0.896024
Closest match: abrdn Inc. with a score of 0.874814093
Closest match: Unum Group with a score of 0.831817687
Closest match: Advent Capital Management, LLC with a score of 0.910662115
Closest match: Aegon Asset Management UK Plc with a score of 0.946337402
Closest match: AIG.COM, Inc. with a score of 0.920619369
Closest match: Bolton Global Capital with a score of 0.885676384
Closest match: Allianz Investment Management SE with a score of 0.927247584
Closest match: Allstate Insurance Company with a score of 0.931931
Closest match: Alantra Corporate Portfolio Advisors with a score of 0.894601941
Closest match: Advanced Capital Gr

In [23]:
# Save the matched vendors to a CSV file
dict_to_csv(matches_dict, 'matched_companies.csv')

Data has been written to matched_companies.csv


In [16]:
vendor_example_dict = {
    "anthony conley": {"id": "rec3iIzIvQQrOrI6D"},
    "Brett Conti LLC": {"id": "recdvz494w9bfFQNi"},
    "Stephanie Hays": {"id": "recHpMGK9u5akKQxb"},
    "Pro Sol License 2": {"id": "recQsM3ScdkiSU56v"},
    "David Zwirner Books": {"id": "recfFTONSXPBsw6Oc"}
}

# Find the closest matches between the source and target vector databases
matches_dict = find_closest_matches('vendors-source', 'vendors-target', vendor_example_dict, pc)

Closest match: Anthony Conley  with a score of 0.954636395
Closest match: Brett Conti with a score of 0.954545856
Closest match: Stephanie Sharp Hays with a score of 0.954265475
Closest match: PROSOL LICENSE 2 with a score of 0.953962445
Closest match: David Zwirner with a score of 0.953940213


In [22]:
vendor_example_dict = {
    "The Montreal Museum of Fine Arts": {"id": "rec6sXgXFizhx2IAA"},
    "New York City Department of Finance": {"id": "recJW3QtRXQg4fVWk"},
    "Sotheby's France": {"id": "rec7PYz5m0PjZVbTf"},
    "Gagosian Gallery International LLC, London branch": {"id": "recrJtywT73ykYHwr"},
    "Nick Callas": {"id": "recw4P2kGP7wtM4OZ"},
    "Moco Museum Barcelona": {"id": "rectgBje56UHXIFjA"},
    "Deep Sync Labs": {"id": "recU3wZMyIqCbo0KA"},
    "Forbes": {"id": "recJ0b4K02eXA3PVV"},
    "Gorvy Levy LLC": {"id": "recKEHlz4Mq0N9qrG"},
    "Julia Nagle Conservation LTD": {"id": "recWkSLbmqfYJBsTH"}
}

# Find the closest matches between the source and target vector databases
matches_dict = find_closest_matches('vendors-source', 'vendors-target', vendor_example_dict, pc)

Closest match: Montreal Museum of Fine Arts with a score of 0.996252656
Closest match: NYC Dept Of Finance Bureau with a score of 0.96269995
Closest match: Sotheby's Paris with a score of 0.959324121
Closest match: Gagosian Gallery London with a score of 0.956362069
Closest match: Nicholas Callas with a score of 0.938691616
Closest match: Modern Contemporary Museum Barcelona with a score of 0.92923528
Closest match: DeepSync with a score of 0.924420416
Closest match: Forbes Media LLC. with a score of 0.916329503
Closest match: Levy Gorvy, New York with a score of 0.910233319
Closest match: Julia Nagle with a score of 0.904301643
